In [2]:
import numpy as np
import random
from numpy.random import randint
from random import random as rnd

In [3]:
import datetime
from datetime import datetime
from datetime import time, timedelta

In [4]:
POIs = ["Pashupatinath Temple", "Boudhanath (Stupa)", "Swayambhunath Stupa", "Tribhuvan Museum","Dakshinkali Temple", "Narayanhiti Palace", "Garden of Dreams"]

In [5]:
def encode(POIs):
    encoded_POI = []
    for index, POI in enumerate(POIs):
        encoded_POI.append(index)
    return encoded_POI

In [6]:
encoded_POI = encode(POIs)

In [7]:
class Individual:
    #Individual generation
    def generate_individual(self, POIs):
        return random.sample(POIs, len(POIs))
    
    def population(self, population_size, POIs):
        return [self.generate_individual(POIs) for x in range(population_size)]
            

In [8]:
def Roulette_wheel_selection(fitness_score):
    Pc = [] #probability of selecting a chromosome
    sum_of_fitness = np.sum(fitness_score)
    for i in fitness_score:
        Pc.append(i/sum_of_fitness)
#     print(Pc)
    return Pc

In [9]:
path_fitness_pair = {} #for global access
def dict_pair(population, fitness_score):
    for i in range(len(population)):
        path_fitness_pair[i] = fitness_score[i]
    return path_fitness_pair

In [10]:
print(path_fitness_pair)

{}


In [11]:
def rank_selected(selected_population):#fitness score of selected population
    key = []
    for i in selected_population:
        index = population.index(i)
        key.append(index)
    #fittest pairing => pairing the selected parents
    #sorting dict by value
    mylist = [path_fitness_pair[i] for i in key] #fitness score of selected parents
    # print(mylist)
    mylist.sort()#ascending order
    # print(mylist)
    mylist = mylist[::-1] # reverse ordering list , descending order
    # print(mylist)
    path_index = [list(path_fitness_pair.values()).index(i) for i in mylist]
    return path_index

In [12]:
from time_period import get_time

In [13]:
opening_time, closing_time = get_time(POIs, 'Monday')

10:00:00
17:00:00
00:00:00
23:59:00
00:00:00
23:59:00
10:00:00
17:00:00
00:00:00
23:59:00
10:00:00
15:30:00
09:00:00
18:30:00


In [14]:
print(opening_time, closing_time)

[datetime.time(10, 0), datetime.time(0, 0), datetime.time(0, 0), datetime.time(10, 0), datetime.time(0, 0), datetime.time(10, 0), datetime.time(9, 0)] [datetime.time(17, 0), datetime.time(23, 59), datetime.time(23, 59), datetime.time(17, 0), datetime.time(23, 59), datetime.time(15, 30), datetime.time(18, 30)]


In [30]:
stay_time = [7200, 7200, 7200, 7200, 7200, 7200, 7200] # 2hrs stay in each poi

In [17]:
from get_time_matrix import get_time_matrix

In [18]:
distance_time = get_time_matrix(POIs)

In [19]:
distance_time

[[60, 660, 1620, 1260, 3240, 840, 780],
 [660, 60, 1920, 1620, 3600, 1140, 1080],
 [1620, 1860, 60, 1020, 2820, 1020, 960],
 [1140, 1500, 1020, 60, 2580, 660, 540],
 [3480, 3840, 2940, 2760, 60, 3060, 2940],
 [720, 1080, 1080, 840, 2940, 60, 180],
 [720, 1080, 900, 720, 2820, 180, 60]]

In [20]:
def waiting_cost(starting_time,i, d = 3600):
    arrival_time = (datetime.combine(datetime.today(),(starting_time))+ timedelta(seconds=d)).time()
    opening = datetime.combine(datetime.today(),(opening_time[i]))
    waiting_time = opening-(datetime.combine(datetime.today(),(starting_time))+ timedelta(seconds=d))
    next_time = (datetime.combine(datetime.today(),starting_time)+ timedelta(seconds=d))+timedelta(seconds = stay_time[i])
#     print(stay_time[i])
#     print(next_time.time())
    return max(waiting_time.total_seconds(), 0), next_time.time(), arrival_time

In [21]:
def delay_cost(leaving_time,i, d = 3600):
    closing = datetime.combine(datetime.today(),(closing_time[i]))
    delay_time = (datetime.combine(datetime.today(),(leaving_time))+ timedelta(seconds=d))-closing
    return max(delay_time.total_seconds(), 0)

In [22]:
#Partially Mapped Crossover
def PMX(offspring1, offspring2, temp1):
    for i in range(len(offspring1)):
        if i < 2 or i>=4: #change the points here
#             print('index',i)
            if temp1[i] not in offspring1:
                offspring1[i] = temp1[i]
#                 print(offspring1[i])
            else:
#                 print(temp1[i] in offspring1[2:4])
                while temp1[i] in offspring1[2:4]:
                    index = offspring1.index(temp1[i])
                    temp1[i] = offspring2[index]
                offspring1[i] = temp1[i]
#                 print(offspring1[i])

In [33]:
class GA:
    def __init__(self, population):
        self.population = population
        
    def fitness_evaluation(self, path):
        count = 0
        fitness_value = 0
        current_time = time(10,0)
        for i in path:
#             print('path',i)
            if count >= 1:
                cost, current_time, arrival_time = waiting_cost(current_time, i, distance_time[j][i])
                delay = delay_cost(current_time, i)
                fitness_value += distance_time[j][i] + cost + delay# distance consideration
    #             print(fitness_value)
#             print('fitness value', fitness_value/3600)
            j = i
            count += 1
            fitness_score = fitness_value/3600
        return 1/fitness_score
    
    #Choose a random number for making it act like a pointer in the wheel
    def selection(self, fitness_score):
        selected = []
        for x in range(len(self.population)//2): #50% selected
            pointer = rnd()
            prob = 0
#             print(pointer)
            path_fitness_pair = dict_pair(self.population, fitness_score)
            for index, i in enumerate(Roulette_wheel_selection(fitness_score)):#roulette wheel
                prob += i #can take out cumulative sum instead see this............
                if prob > pointer:
#                     print(index)
                    selected.append(self.population[index])
#                     print('fitness_score',path_fitness_pair[index])
                    break
#             print(selected)
        return selected  
    
    #fittest pairing
    def pairing(self, selected_population):
        parents = [[selected_population[x],selected_population[x+1]] for x in range(len(selected_population)-1)]
        return parents
    
    #crossover
    #two point crossover
    def two_point_crossover(self, parent1, parent2):
    #     pivot_point1 = randint(1, len(selected)-1)
    #     pivot_point2 = randint(1, len(selected))
    #     while(pivot_point2 <= pivot_point1):
    #         pivot_point2 = randint(1, len(selected))
    #     print(pivot_point1, pivot_point2)
    #     print(parent1, parent2)

        offspring1 = [-1]*len(parent1)
        offspring2 = [-1]*len(parent1)
        offspring1[2:4] = parent2[2:4]
        offspring2[2:4] = parent1[2:4]

    #     print(parent1, parent2)
        temp1 = parent1.copy()
        temp2 = parent2.copy() #copy garena bhane parent ma ni modification aauxa but why???

    #     print(offspring1)
        PMX(offspring1, offspring2, temp1)
        PMX(offspring2, offspring1, temp2)

        return [offspring1, offspring2]
    #     return offspring1, offspring2 =>list of tuples of list
    #     print(parent1, parent2)
    
    #Scramble mutation : choose two points then bichko genes lai shuffle garne
    def individual_for_mutation(self, mutation_rate = 0.1):
        individual_to_mutate = []
        for x in range(round(len(self.population)*mutation_rate)):
            individual_to_mutate.append(self.population[randint(0, len(self.population)-1)])
        return individual_to_mutate
    
    def scramble_mutation(self, individual):
        p1 = randint(0, len(individual)-1)
        p2 = randint(0, len(individual))
        while p1 >= p2:
            p2 = randint(0, len(individual))
        c2 = individual[p1:p2]
        random.shuffle(c2)
        for i in c2:
            individual[p1] = i
            p1 +=1
        return individual

In [34]:
def best_solution(populaiton, fitness_score):
    best_fittest = np.max(fitness_score)
    index = fitness_score.index(best_fittest)
    best_individual = population[index]
    return best_fittest, best_individual

In [35]:
i = Individual()
initial_population = i.population(20, encoded_POI)

In [36]:
population = initial_population

In [37]:
def run(population):
    g = GA(population)
    fitness_score = [g.fitness_evaluation(individual) for individual in population]
    best_fittest, best_individual = best_solution(population, fitness_score)
    selected = g.selection(fitness_score)
#     print(selected)
    path_index = rank_selected(selected)
#     print(path_index)
    paired_parents = g.pairing(selected)
#     print(paired_parents)
    offsprings = []
    for i in paired_parents:
        offsprings.append(g.two_point_crossover(i[0], i[1]))
#     print(offsprings)
    next_population = []
    for i in offsprings:
        next_population.extend(i)
#     print('nextpopulation',next_population)
    individual_to_mutate = g.individual_for_mutation()
#     print('individual_to_mutate',individual_to_mutate)
    for x in individual_to_mutate:
        next_population.append(g.scramble_mutation(x))
#     print('nextpopulation',next_population)
#     initial_population = next_population
    return best_fittest, best_individual, next_population

In [38]:
generation = 100
for i in range(generation):
    best_fittest, best_individual, next_population = run(population)
    print('Generation',i,':', 'Best Fittest:',best_fittest," Best Individual:",best_individual)
    while len(next_population) < 20:
        print(len(next_population))
        print(population[randint(0, len(population))])
        next_population.append(population[randint(0, len(population))])
    population = next_population
    
route = []
for i in best_individual:
    route.append(POIs[i])
print(route)

Generation 0 : Best Fittest: 0.3243243243243243  Best Individual: [5, 0, 3, 4, 6, 1, 2]
Generation 1 : Best Fittest: 0.3076923076923077  Best Individual: [0, 4, 3, 6, 1, 2, 5]
Generation 2 : Best Fittest: 0.3314917127071823  Best Individual: [0, 6, 3, 4, 1, 2, 5]
Generation 3 : Best Fittest: 0.35714285714285715  Best Individual: [5, 0, 3, 6, 2, 4, 1]
Generation 4 : Best Fittest: 0.35714285714285715  Best Individual: [5, 0, 3, 6, 2, 4, 1]
Generation 5 : Best Fittest: 0.3278688524590164  Best Individual: [0, 2, 3, 6, 4, 1, 5]
Generation 6 : Best Fittest: 0.3278688524590164  Best Individual: [0, 2, 3, 6, 4, 1, 5]
Generation 7 : Best Fittest: 0.3488372093023256  Best Individual: [0, 6, 3, 2, 4, 1, 5]
Generation 8 : Best Fittest: 0.3278688524590164  Best Individual: [0, 2, 3, 6, 4, 1, 5]
Generation 9 : Best Fittest: 0.4054054054054054  Best Individual: [5, 0, 3, 6, 2, 1, 4]
Generation 10 : Best Fittest: 0.4054054054054054  Best Individual: [5, 0, 3, 6, 2, 1, 4]
Generation 11 : Best Fittest:

Generation 93 : Best Fittest: 0.4411764705882353  Best Individual: [3, 0, 5, 6, 1, 2, 4]
Generation 94 : Best Fittest: 0.4411764705882353  Best Individual: [3, 0, 5, 6, 1, 2, 4]
Generation 95 : Best Fittest: 0.45112781954887216  Best Individual: [3, 5, 0, 6, 1, 2, 4]
Generation 96 : Best Fittest: 0.45112781954887216  Best Individual: [3, 5, 0, 6, 1, 2, 4]
Generation 97 : Best Fittest: 0.45112781954887216  Best Individual: [3, 5, 0, 6, 1, 2, 4]
Generation 98 : Best Fittest: 0.45112781954887216  Best Individual: [3, 5, 0, 6, 1, 2, 4]
Generation 99 : Best Fittest: 0.45112781954887216  Best Individual: [3, 5, 0, 6, 1, 2, 4]
['Tribhuvan Museum', 'Narayanhiti Palace', 'Pashupatinath Temple', 'Garden of Dreams', 'Boudhanath (Stupa)', 'Swayambhunath Stupa', 'Dakshinkali Temple']


In [40]:
current_time = time(10,0)
print('Starts at:', current_time)
count = 0
for place in route:
    i = POIs.index(place)
    if count >= 1:
        cost, current_time, arrival_time = waiting_cost(current_time, i, distance_time[j][i])
        print('Arrive ',place, "at:", arrival_time)
        print('Leave ',place," at:", current_time)
        
    else:
        cost, current_time, arrival_time = waiting_cost(current_time, i)
        print('Arrive ',place, " at:", arrival_time)
        print('Leave ',place," at:", current_time)
    j = i
    count += 1


Starts at: 10:00:00
Arrive  Tribhuvan Museum  at: 11:00:00
Leave  Tribhuvan Museum  at: 13:00:00
Arrive  Narayanhiti Palace at: 13:11:00
Leave  Narayanhiti Palace  at: 15:11:00
Arrive  Pashupatinath Temple at: 15:23:00
Leave  Pashupatinath Temple  at: 17:23:00
Arrive  Garden of Dreams at: 17:36:00
Leave  Garden of Dreams  at: 19:36:00
Arrive  Boudhanath (Stupa) at: 19:54:00
Leave  Boudhanath (Stupa)  at: 21:54:00
Arrive  Swayambhunath Stupa at: 22:26:00
Leave  Swayambhunath Stupa  at: 00:26:00
Arrive  Dakshinkali Temple at: 01:13:00
Leave  Dakshinkali Temple  at: 03:13:00


In [34]:
population

[[3, 1, 0, 4, 2],
 [3, 1, 4, 2, 0],
 [1, 0, 3, 4, 2],
 [0, 1, 3, 4, 2],
 [0, 2, 3, 1, 4],
 [2, 0, 3, 1, 4],
 [0, 3, 2, 4, 1],
 [4, 0, 2, 3, 1],
 [0, 1, 3, 4, 2],
 [3, 2, 0, 4, 1],
 [1, 3, 2, 4, 0],
 [4, 1, 3, 2, 0],
 [2, 1, 4, 3, 0],
 [3, 0, 2, 1, 4],
 [0, 4, 1, 3, 2],
 [0, 3, 2, 1, 4],
 [1, 4, 2, 0, 3],
 [3, 4, 0, 1, 2],
 [2, 1, 3, 0, 4],
 [2, 0, 4, 3, 1],
 [1, 2, 3, 4, 0],
 [3, 1, 4, 0, 2],
 [1, 2, 3, 4, 0],
 [3, 0, 1, 2, 4],
 [2, 4, 0, 3, 1],
 [2, 4, 3, 0, 1],
 [2, 1, 0, 3, 4],
 [3, 4, 1, 2, 0],
 [4, 1, 3, 0, 2],
 [2, 1, 4, 0, 3]]